In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
def crawl_data(comp_standings_urls, year_max, year_min, output_filename, num_eg=0):
    years = list(range(year_max, year_min, -1))
    all_matches = []
    eg = int(0)
    num_eg_ = int(num_eg/10)
    print(years)
    for (standings_url, comp_cur) in comp_standings_urls:
        for year in years:
            data = requests.get(standings_url)
            soup = BeautifulSoup(data.text)
            standings_table = soup.select('table.stats_table')[0]

            links = [l.get("href") for l in standings_table.find_all('a')]
            links = [l for l in links if '/squads/' in l]
            team_urls = [f"https://fbref.com{l}" for l in links]
            
            previous_season = soup.select("a.prev")[0].get("href")
            standings_url = f"https://fbref.com{previous_season}"
            
            for team_url in team_urls:
                if num_eg_ > 0 and eg == num_eg_:
                    match_df = pd.concat(all_matches)
                    match_df.columns = [c.lower() for c in match_df.columns]
                    match_df = match_df[0:num_eg-1]
                    match_df.to_csv(output_filename)
                    return
                team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
                data = requests.get(team_url)
                matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
                soup = BeautifulSoup(data.text)
                links = [l.get("href") for l in soup.find_all('a')]
                links = [l for l in links if l and 'all_comps/shooting/' in l]
                data = requests.get(f"https://fbref.com{links[0]}")
                shooting = pd.read_html(data.text, match="Shooting")[0]
                shooting.columns = shooting.columns.droplevel()
                try:
                    team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
                except ValueError:
                    continue
                team_data = team_data[team_data["Comp"] == comp_cur]

                team_data["Season"] = year
                team_data["Team"] = team_name
                all_matches.append(team_data)
                eg += 1
                time.sleep(1)
    
    match_df = pd.concat(all_matches)
    match_df.columns = [c.lower() for c in match_df.columns]
    match_df.to_csv(output_filename)
    

In [2]:
standings_Premier_Leagure_url = ("https://fbref.com/en/comps/9/Premier-League-Stats", "Premier League")
standings_LaLiga_url = ("https://fbref.com/en/comps/12/La-Liga-Stats", "La Liga")
standings_Bundesliga_url = ("https://fbref.com/en/comps/20/Bundesliga-Stats", "Bundesliga")


In [4]:
urls_10000 = [standings_Premier_Leagure_url, standings_LaLiga_url, standings_Bundesliga_url]
urls_1000 = [standings_Premier_Leagure_url]

### Dữ liệu cho 10 mẫu

In [26]:
crawl_data(urls_1000, 2023, 2021, "10_eg.csv", 10)

[2023, 2022]


### Dữ liệu cho 1000 mẫu

In [5]:
crawl_data(urls_1000, 2023, 2021, "1000_raw_data.csv")

[2023, 2022]


### Dữ liệu cho 10000 mẫu

In [6]:
crawl_data(urls_10000, 2023, 2018, "10000_raw_data.csv") 

[2023, 2022, 2021, 2020, 2019]
